# Demostración

In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import datetime
import re

import sys

from tensorflow import keras

In [2]:
model = keras.models.load_model("modelo_pronosticador_regina.h5")

file = open("norm.params", "rb")
train_mean, train_std, wind_dic = pickle.load(file)
file.close()

In [3]:
drop = [0, 1, 3, 4, 12, 13, 15, 14, 9, 20, 21, 27, 31, 32]

def transform_date(date, date_format):
    date = pd.Timestamp.timestamp(pd.Timestamp(datetime.datetime.strptime(date, date_format)))
    
    day = 24*60*60
    year = 365.2425 * day
    
    day_sin  = np.sin(date * (2 * np.pi / day))
    day_cos  = np.cos(date * (2 * np.pi / day))
    year_sin = np.sin(date * (2 * np.pi / year))
    year_cos = np.cos(date * (2 * np.pi / year))
    
    return [day_sin, day_cos, year_sin, year_cos]

def convert(data, regex = "\s+", date_format = "%d.%m.%y %H:%M"):
    data = data.splitlines()
    conv_data = np.zeros((0, 23))
    
    for sample in data:
        sample = re.split(regex, sample)
        date = sample[0] + " " + sample[1]
        sample[8]  = wind_dic[sample[8]]
        sample[11] = wind_dic[sample[11]]
        sample = np.delete(sample, drop)
        sample = sample.astype(float)
        #sample[11] /= 1000
        #sample[12] /= 1000
        sample[0] -= 2
        
        sample = np.concatenate((sample, transform_date(date, date_format)))
        
        sample = (sample-train_mean)/train_std

        conv_data = np.vstack((conv_data, sample))
    
    conv_data = np.reshape(conv_data, (1, 90, 23))
    return conv_data

In [4]:
x1 = convert("""22.10.20 21:10	17.6	17.7	17.6	68	11.6	0.0	---	0.00	0.0	---	17.6	17.4	17.4	---	763.9	0.00	0.0	0	0.00	0	5	0	21.9	46	9.8	21.1	0.00	234	1	100.0	10
22.10.20 21:20	17.5	17.6	17.5	69	11.7	0.0	SE	0.00	1.6	SE	17.5	17.3	17.3	---	764.1	0.00	0.0	0	0.00	0	6	0	21.9	46	9.7	21.0	0.00	234	1	100.0	10
22.10.20 21:30	17.6	17.6	17.5	68	11.6	0.0	NNW	0.00	8.0	NNW	17.6	17.3	17.3	---	764.1	0.00	0.0	0	0.00	0	5	0	21.8	46	9.6	20.8	0.00	233	1	100.0	10
22.10.20 21:40	17.7	17.7	17.6	68	11.7	0.0	NNW	0.00	3.2	NNW	17.7	17.5	17.5	---	764.0	0.00	0.0	0	0.00	0	5	0	21.8	46	9.6	20.8	0.00	234	1	100.0	10
22.10.20 21:50	17.6	17.6	17.6	68	11.6	0.0	N	0.00	6.4	NNW	17.6	17.3	17.3	---	764.0	0.00	0.0	0	0.00	0	5	0	21.7	46	9.6	20.7	0.00	231	1	100.0	10
22.10.20 22:00	17.4	17.6	17.4	68	11.5	0.0	WNW	0.00	4.8	NNE	17.4	17.2	17.2	---	764.0	0.00	0.0	0	0.00	0	6	0	21.7	46	9.6	20.7	0.03	233	1	100.0	10
22.10.20 22:10	17.3	17.4	17.2	69	11.5	0.0	E	0.00	6.4	E	17.3	17.1	17.1	---	764.2	0.00	0.0	0	0.00	0	7	0	21.6	46	9.5	20.6	0.00	234	1	100.0	10
22.10.20 22:20	17.1	17.2	17.1	70	11.6	0.0	---	0.00	0.0	---	17.1	16.9	16.9	---	764.2	0.00	0.0	0	0.00	0	8	0	21.6	46	9.5	20.6	0.00	234	1	100.0	10
22.10.20 22:30	17.0	17.1	16.9	70	11.5	0.0	SSE	0.00	3.2	SSE	17.0	16.8	16.8	---	764.4	0.00	0.0	0	0.00	0	9	0	21.6	46	9.4	20.5	0.00	234	1	100.0	10
22.10.20 22:40	16.8	16.9	16.8	71	11.5	0.0	ENE	0.00	1.6	ENE	16.8	16.6	16.6	---	764.4	0.00	0.0	0	0.00	0	10	0	21.5	46	9.4	20.4	0.00	232	1	100.0	10
22.10.20 22:50	16.6	16.8	16.5	71	11.3	0.0	---	0.00	0.0	---	16.6	16.4	16.4	---	764.4	0.00	0.0	0	0.00	0	12	0	21.4	46	9.3	20.4	0.00	234	1	100.0	10
22.10.20 23:00	16.3	16.4	16.3	72	11.3	0.0	E	0.00	1.6	E	16.3	16.1	16.1	---	764.4	0.00	0.0	0	0.00	0	14	0	21.4	46	9.3	20.4	0.03	234	1	100.0	10
22.10.20 23:10	16.1	16.2	16.1	73	11.3	0.0	---	0.00	0.0	---	16.1	15.9	15.9	---	764.5	0.00	0.0	0	0.00	0	15	0	21.3	46	9.2	20.2	0.00	235	1	100.0	10
22.10.20 23:20	16.0	16.1	15.9	74	11.4	0.0	---	0.00	0.0	---	16.0	15.8	15.8	---	764.5	0.00	0.0	0	0.00	0	16	0	21.3	46	9.2	20.2	0.00	234	1	100.0	10
22.10.20 23:30	15.9	15.9	15.9	74	11.3	1.6	SE	0.27	8.0	SE	15.9	15.7	15.7	---	764.5	0.00	0.0	0	0.00	0	17	0	21.2	47	9.5	20.2	0.00	234	1	100.0	10
22.10.20 23:40	15.9	15.9	15.8	75	11.5	0.0	NE	0.00	4.8	NNW	15.9	15.7	15.7	---	764.5	0.00	0.0	0	0.00	0	17	0	21.2	47	9.5	20.2	0.00	234	1	100.0	10
22.10.20 23:50	15.6	15.8	15.5	76	11.4	0.0	E	0.00	3.2	E	15.6	15.4	15.4	---	764.5	0.00	0.0	0	0.00	0	19	0	21.2	47	9.4	20.1	0.00	233	1	100.0	10
23.10.20 00:00	15.4	15.5	15.3	77	11.4	0.0	ENE	0.00	3.2	ENE	15.4	15.2	15.2	---	764.4	0.00	0.0	0	0.00	0	20	0	21.1	47	9.3	19.9	0.03	234	1	100.0	10
23.10.20 0:10	15.2	15.3	15.1	78	11.4	0.0	ESE	0.00	4.8	ESE	15.2	15.0	15.0	---	764.3	0.00	0.0	0	0.00	0	22	0	21.1	47	9.3	19.9	0.00	233	1	100.0	10
23.10.20 0:20	15.0	15.1	14.9	79	11.4	0.0	SE	0.00	4.8	E	15.0	14.8	14.8	---	764.4	0.00	0.0	0	0.00	0	23	0	20.9	47	9.2	19.9	0.00	234	1	100.0	10
23.10.20 0:30	14.8	14.9	14.8	79	11.2	0.0	SE	0.00	4.8	SE	14.8	14.7	14.7	---	764.3	0.00	0.0	0	0.00	0	24	0	20.9	47	9.2	19.9	0.00	234	1	100.0	10
23.10.20 0:40	14.7	14.8	14.7	80	11.3	0.0	ENE	0.00	4.8	NE	14.7	14.6	14.6	---	764.4	0.00	0.0	0	0.00	0	25	0	20.9	47	9.2	19.8	0.00	235	1	100.0	10
23.10.20 0:50	14.6	14.7	14.5	80	11.1	0.0	SSE	0.00	4.8	SSW	14.6	14.4	14.4	---	764.2	0.00	0.0	0	0.00	0	26	0	20.8	47	9.1	19.8	0.00	234	1	100.0	10
23.10.20 1:00	14.4	14.5	14.3	81	11.2	0.0	ESE	0.00	4.8	ESE	14.4	14.3	14.3	---	764.2	0.00	0.0	0	0.00	0	27	0	20.8	47	9.0	19.8	0.03	234	1	100.0	10
23.10.20 1:10	14.3	14.3	14.3	82	11.3	0.0	ESE	0.00	4.8	ESE	14.3	14.2	14.2	---	764.2	0.00	0.0	0	0.00	0	28	0	20.7	48	9.3	19.7	0.00	233	1	100.0	10
23.10.20 1:20	14.2	14.3	14.1	82	11.1	0.0	SE	0.00	4.8	ENE	14.2	14.1	14.1	---	764.3	0.00	0.0	0	0.00	0	29	0	20.6	48	9.2	19.7	0.00	233	1	100.0	10
23.10.20 1:30	14.1	14.1	14.0	82	11.0	0.0	---	0.00	0.0	---	14.1	13.9	13.9	---	764.2	0.00	0.0	0	0.00	0	30	0	20.6	48	9.2	19.7	0.00	234	1	100.0	10
23.10.20 1:40	14.0	14.0	13.9	83	11.2	0.0	SE	0.00	3.2	SE	14.0	13.9	13.9	---	764.0	0.00	0.0	0	0.00	0	30	0	20.5	48	9.1	19.6	0.00	235	1	100.0	10
23.10.20 1:50	13.9	13.9	13.9	83	11.1	0.0	NNE	0.00	9.7	ESE	13.9	13.8	13.8	---	764.0	0.00	0.0	0	0.00	0	30	0	20.4	48	9.0	19.5	0.00	234	1	100.0	10
23.10.20 2:00	13.9	13.9	13.8	84	11.2	0.0	NE	0.00	3.2	NE	13.9	13.8	13.8	---	763.9	0.00	0.0	0	0.00	0	31	0	20.3	48	9.0	19.4	0.03	234	1	100.0	10
23.10.20 2:10	13.8	13.8	13.7	84	11.1	0.0	---	0.00	0.0	---	13.8	13.7	13.7	---	763.9	0.00	0.0	0	0.00	0	32	0	20.3	48	8.9	19.4	0.00	234	1	100.0	10
23.10.20 2:20	13.7	13.7	13.7	84	11.1	0.0	WSW	0.00	6.4	ESE	13.7	13.7	13.7	---	763.8	0.00	0.0	0	0.00	0	32	0	20.2	48	8.9	19.4	0.00	233	1	100.0	10
23.10.20 2:30	13.7	13.7	13.7	85	11.2	1.6	NE	0.27	9.7	N	13.7	13.7	13.7	---	763.8	0.00	0.0	0	0.00	0	32	0	20.2	48	8.8	19.3	0.00	234	1	100.0	10
23.10.20 2:40	13.7	13.8	13.7	85	11.2	1.6	NNE	0.27	8.0	NW	13.7	13.7	13.7	---	763.9	0.00	0.0	0	0.00	0	32	0	20.1	49	9.0	19.3	0.00	233	1	100.0	10
23.10.20 2:50	13.7	13.8	13.7	85	11.2	1.6	E	0.27	8.0	NNW	13.7	13.6	13.6	---	763.6	0.00	0.0	0	0.00	0	32	0	20.0	49	9.0	19.2	0.00	234	1	100.0	10
23.10.20 3:00	13.7	13.7	13.6	85	11.2	0.0	---	0.00	0.0	---	13.7	13.6	13.6	---	763.7	0.00	0.0	0	0.00	0	32	0	19.9	49	8.9	19.2	0.03	234	1	100.0	10
23.10.20 3:10	13.6	13.6	13.5	85	11.1	0.0	ENE	0.00	6.4	NE	13.6	13.5	13.5	---	763.6	0.00	0.0	0	0.00	0	33	0	19.8	49	8.8	19.1	0.00	234	1	100.0	10
23.10.20 3:20	13.4	13.5	13.3	85	11.0	0.0	W	0.00	3.2	NE	13.4	13.4	13.4	---	763.5	0.00	0.0	0	0.00	0	34	0	19.8	49	8.8	19.0	0.00	234	1	100.0	10
23.10.20 3:30	13.2	13.3	13.2	86	10.9	1.6	ESE	0.27	4.8	SE	13.2	13.2	13.2	---	763.5	0.00	0.0	0	0.00	0	35	0	19.7	49	8.7	18.9	0.00	234	1	100.0	10
23.10.20 3:40	13.3	13.3	13.3	87	11.2	0.0	NW	0.00	3.2	NW	13.3	13.3	13.3	---	763.4	0.00	0.0	0	0.00	0	35	0	19.7	49	8.6	18.8	0.00	233	1	100.0	10
23.10.20 3:50	13.3	13.4	13.3	86	11.0	0.0	N	0.00	6.4	NNE	13.3	13.3	13.3	---	763.4	0.00	0.0	0	0.00	0	35	0	19.6	49	8.5	18.7	0.00	235	1	100.0	10
23.10.20 4:00	13.2	13.3	13.2	87	11.1	0.0	---	0.00	0.0	---	13.2	13.2	13.2	---	763.4	0.00	0.0	0	0.00	0	35	0	19.4	50	8.7	18.7	0.00	234	1	100.0	10
23.10.20 4:10	13.2	13.3	13.2	87	11.1	0.0	N	0.00	3.2	N	13.2	13.2	13.2	---	763.4	0.00	0.0	0	0.00	0	35	0	19.4	50	8.7	18.7	0.00	233	1	100.0	10
23.10.20 4:20	13.3	13.3	13.3	87	11.2	0.0	---	0.00	0.0	---	13.3	13.3	13.3	---	763.5	0.00	0.0	0	0.00	0	35	0	19.4	50	8.7	18.6	0.00	234	1	100.0	10
23.10.20 4:30	13.3	13.3	13.2	87	11.2	0.0	---	0.00	0.0	---	13.3	13.3	13.3	---	763.4	0.00	0.0	0	0.00	0	35	0	19.3	50	8.6	18.4	0.00	232	1	100.0	10
23.10.20 4:40	13.3	13.3	13.2	88	11.3	0.0	---	0.00	0.0	---	13.3	13.3	13.3	---	763.5	0.00	0.0	0	0.00	0	35	0	19.3	50	8.6	18.4	0.00	233	1	100.0	10
23.10.20 4:50	13.4	13.4	13.3	88	11.4	0.0	---	0.00	0.0	---	13.4	13.4	13.4	---	763.5	0.00	0.0	0	0.00	0	34	0	19.2	50	8.5	18.4	0.00	234	1	100.0	10
23.10.20 5:00	13.5	13.6	13.4	88	11.6	0.0	NNW	0.00	4.8	NNW	13.5	13.5	13.5	---	763.7	0.00	0.0	0	0.00	0	34	0	19.2	51	8.8	18.4	0.00	235	1	100.0	10
23.10.20 5:10	13.7	13.8	13.6	88	11.8	0.0	ENE	0.00	6.4	NE	13.7	13.7	13.7	---	763.7	0.00	0.0	0	0.00	0	32	0	19.1	51	8.7	18.3	0.00	234	1	100.0	10
23.10.20 5:20	13.8	13.9	13.8	88	11.9	1.6	N	0.27	4.8	N	13.8	13.8	13.8	---	763.6	0.00	0.0	0	0.00	0	31	0	19.0	51	8.6	18.2	0.00	233	1	100.0	10
23.10.20 5:30	13.9	14.0	13.9	87	11.8	1.6	N	0.27	6.4	NNE	13.9	13.9	13.9	---	763.6	0.00	0.0	0	0.00	0	30	0	19.0	51	8.6	18.2	0.00	234	1	100.0	10
23.10.20 5:40	14.0	14.1	14.0	87	11.9	1.6	N	0.27	6.4	N	14.0	14.0	14.0	---	763.5	0.00	0.0	0	0.00	0	30	0	18.9	51	8.6	18.1	0.00	234	1	100.0	10
23.10.20 5:50	14.1	14.1	14.0	87	11.9	0.0	N	0.00	3.2	N	14.1	14.1	14.1	---	763.5	0.00	0.0	0	0.00	0	30	0	18.9	51	8.6	18.1	0.00	234	1	100.0	10
23.10.20 6:00	14.1	14.1	14.1	87	11.9	0.0	N	0.00	3.2	N	14.1	14.1	14.1	---	763.5	0.00	0.0	0	0.00	0	30	0	18.9	51	8.5	18.1	0.00	234	1	100.0	10
23.10.20 6:10	14.1	14.1	14.1	87	12.0	0.0	---	0.00	0.0	---	14.1	14.1	14.1	---	763.5	0.00	0.0	0	0.00	0	29	0	18.8	51	8.5	18.0	0.00	235	1	100.0	10
23.10.20 6:20	14.1	14.2	14.1	87	12.0	0.0	---	0.00	0.0	---	14.1	14.1	14.1	---	763.4	0.00	0.0	0	0.00	0	29	0	18.8	52	8.8	18.1	0.00	233	1	100.0	10
23.10.20 6:30	14.2	14.2	14.1	87	12.0	0.0	---	0.00	0.0	---	14.2	14.2	14.2	---	763.4	0.00	0.0	0	0.00	0	29	0	18.8	52	8.8	18.1	0.00	233	1	100.0	10
23.10.20 6:40	14.2	14.3	14.2	87	12.1	0.0	---	0.00	0.0	---	14.2	14.2	14.2	---	763.5	0.00	0.0	3	0.04	5	29	0	18.8	52	8.7	18.0	0.00	234	1	100.0	10
23.10.20 6:50	14.3	14.4	14.3	87	12.2	0.0	N	0.00	1.6	N	14.3	14.3	14.3	---	763.5	0.00	0.0	9	0.13	11	28	0	18.8	52	8.7	18.0	0.00	234	1	100.0	10
23.10.20 7:00	14.4	14.5	14.4	87	12.3	0.0	---	0.00	0.0	---	14.4	14.4	14.4	---	763.5	0.00	0.0	14	0.20	18	27	0	18.8	52	8.7	18.0	0.00	234	1	100.0	10
23.10.20 7:10	14.6	14.7	14.5	87	12.5	0.0	N	0.00	3.2	N	14.6	14.6	14.6	---	763.5	0.00	0.0	23	0.33	26	26	0	18.8	52	8.7	18.0	0.00	233	1	100.0	10
23.10.20 7:20	14.7	14.7	14.7	87	12.6	0.0	NNE	0.00	6.4	N	14.7	14.7	14.7	---	763.5	0.00	0.0	31	0.44	32	25	0	18.8	52	8.7	18.0	0.00	234	1	100.0	10
23.10.20 7:30	14.8	14.8	14.7	86	12.5	1.6	NNE	0.27	8.0	NNW	14.8	14.8	14.8	---	763.5	0.00	0.0	34	0.49	40	25	0	18.7	52	8.7	17.9	0.00	234	1	100.0	10
23.10.20 7:40	14.9	14.9	14.8	86	12.6	1.6	NNW	0.27	6.4	NNW	14.9	14.9	14.9	---	763.6	0.00	0.0	44	0.63	47	24	0	18.7	52	8.6	17.9	0.00	233	1	100.0	10
23.10.20 7:50	15.0	15.1	14.9	86	12.7	1.6	NNE	0.27	8.0	NNW	15.0	15.0	15.0	---	763.6	0.00	0.0	52	0.75	58	23	0	18.7	52	8.6	17.9	0.00	233	1	100.0	10
23.10.20 8:00	15.1	15.2	15.0	85	12.6	1.6	N	0.27	6.4	NE	15.1	15.1	15.1	---	763.6	0.00	0.0	63	0.90	67	23	0	18.7	52	8.6	17.9	0.03	234	1	100.0	10
23.10.20 8:10	15.2	15.3	15.1	85	12.7	0.0	N	0.00	8.0	NNW	15.2	15.2	15.2	---	763.6	0.00	0.0	68	0.97	76	22	0	18.7	52	8.6	17.9	0.00	232	1	100.0	10
23.10.20 8:20	15.3	15.4	15.3	85	12.8	1.6	NW	0.27	4.8	NW	15.3	15.4	15.4	---	763.6	0.00	0.0	78	1.12	81	21	0	18.7	52	8.6	17.9	0.00	235	1	100.0	10
23.10.20 8:30	15.4	15.5	15.4	84	12.8	1.6	WNW	0.27	8.0	N	15.4	15.4	15.4	---	763.5	0.00	0.0	84	1.20	97	20	0	18.7	52	8.6	17.9	0.00	234	1	100.0	10
23.10.20 8:40	15.5	15.6	15.5	84	12.8	1.6	N	0.27	6.4	N	15.5	15.6	15.6	---	763.6	0.00	0.0	96	1.38	100	20	0	18.7	52	8.6	17.9	0.00	233	1	100.0	10
23.10.20 8:50	15.7	15.7	15.6	84	13.0	1.6	N	0.27	6.4	N	15.7	15.7	15.7	---	763.6	0.00	0.0	106	1.52	120	19	0	18.7	52	8.6	17.9	0.00	233	1	100.0	10
23.10.20 9:00	15.8	15.9	15.7	83	12.9	1.6	NNW	0.27	6.4	N	15.8	15.9	15.9	---	763.6	0.00	0.0	128	1.83	135	17	0	18.7	52	8.6	17.9	0.05	234	1	100.0	10
23.10.20 9:10	16.1	16.2	15.9	83	13.2	1.6	NNW	0.27	6.4	NNW	16.1	16.1	16.1	---	763.6	0.00	0.0	143	2.05	151	16	0	18.7	52	8.6	17.9	0.00	233	1	100.0	10
23.10.20 9:20	16.3	16.5	16.2	82	13.3	1.6	NNE	0.27	6.4	ESE	16.3	16.4	16.4	---	763.6	0.00	0.0	166	2.38	183	14	0	18.7	52	8.6	17.9	0.00	234	1	100.0	10
23.10.20 9:30	16.7	16.8	16.5	81	13.4	1.6	NE	0.27	6.4	NE	16.7	16.8	16.8	---	763.5	0.00	0.0	190	2.72	202	12	0	18.7	52	8.6	17.9	0.00	233	1	100.0	10
23.10.20 9:40	16.9	17.1	16.8	80	13.5	1.6	NW	0.27	6.4	NNW	16.9	17.1	17.1	---	763.4	0.00	0.0	205	2.94	213	10	0	18.8	52	8.7	18.0	0.00	233	1	100.0	10
23.10.20 9:50	17.3	17.4	17.1	79	13.6	1.6	NE	0.27	8.0	NNE	17.3	17.4	17.4	---	763.5	0.00	0.0	238	3.41	262	7	0	18.8	52	8.7	18.0	0.00	234	1	100.0	10
23.10.20 10:00	17.6	17.8	17.4	77	13.5	1.6	NNW	0.27	8.0	NNW	17.6	17.7	17.7	---	763.4	0.00	0.0	296	4.24	311	5	0	18.8	52	8.7	18.0	0.13	234	1	100.0	10
23.10.20 10:10	17.9	18.0	17.8	76	13.6	3.2	NW	0.54	9.7	NE	17.9	18.0	18.0	---	763.4	0.00	0.0	326	4.67	343	3	0	18.8	52	8.8	18.1	0.00	233	1	100.0	10
23.10.20 10:20	18.1	18.2	18.0	78	14.2	1.6	NNW	0.27	9.7	NNW	18.1	18.3	18.3	---	763.3	0.00	0.0	347	4.97	373	2	0	18.8	52	8.8	18.1	0.00	234	1	100.0	10
23.10.20 10:30	18.5	18.7	18.2	74	13.8	1.6	NNE	0.27	8.0	NE	18.5	18.7	18.7	---	763.2	0.00	0.0	408	5.85	446	0	1	18.9	52	8.8	18.1	0.00	235	1	100.0	10
23.10.20 10:40	18.7	18.7	18.7	73	13.8	3.2	NNE	0.54	11.3	N	18.7	18.9	18.9	---	763.1	0.00	0.0	439	6.29	513	0	3	18.9	52	8.9	18.2	0.00	234	1	100.0	10
23.10.20 10:50	18.8	18.9	18.7	74	14.1	3.2	N	0.54	11.3	NE	18.8	19.1	19.1	---	763.1	0.00	0.0	462	6.62	540	0	3	19.0	52	8.9	18.2	0.00	232	1	100.0	10
23.10.20 11:00	19.5	20.0	18.9	70	13.9	1.6	N	0.27	9.7	WNW	19.5	19.7	19.7	---	762.9	0.00	0.0	946	13.56	1048	0	8	19.1	52	9.0	18.3	0.28	234	1	100.0	10
23.10.20 11:10	20.2	20.4	20.0	70	14.5	3.2	NE	0.54	9.7	NW	20.2	20.4	20.4	---	762.8	0.00	0.0	890	12.76	1002	0	13	19.2	53	9.4	18.5	0.00	234	1	100.0	10
23.10.20 11:20	20.7	20.7	20.4	68	14.5	1.6	N	0.27	11.3	N	20.7	20.7	20.7	---	762.8	0.00	0.0	814	11.67	881	0	16	19.2	53	9.4	18.6	0.00	233	1	100.0	10
23.10.20 11:30	20.9	21.1	20.7	68	14.8	1.6	E	0.27	8.0	SE	20.9	20.9	20.9	---	762.6	0.00	0.0	798	11.44	824	0	18	19.4	54	9.8	18.8	0.00	235	1	100.0	10
23.10.20 11:40	21.2	21.3	21.1	68	15.1	1.6	E	0.27	9.7	ENE	21.2	21.2	21.2	---	762.4	0.00	0.0	830	11.90	837	0	20	19.4	54	9.9	18.9	0.00	234	1	100.0	10
23.10.20 11:50	21.4	21.6	21.3	66	14.8	3.2	N	0.54	14.5	NNE	21.4	21.4	21.4	---	762.5	0.00	0.0	843	12.08	851	0	22	19.6	54	10.0	19.1	0.00	234	1	100.0	10
23.10.20 12:00	21.7	21.8	21.6	65	14.9	4.8	NNE	0.80	14.5	NNE	21.7	21.7	21.7	---	762.4	0.00	0.0	856	12.27	860	0	24	19.7	54	10.1	19.2	0.48	233	1	100.0	10
""")

pred = model.predict(x1)

print(pred)

[[0.2993223]]


In [5]:
inta_regex  = "\s+(?!$)"
inta_format = "%d/%m/%y %H:%M"

x = convert("""15/03/22   1:10   12.3   12.3   12.3    86   10.0   0.0   ---   0.00   0.0   ---   12.3   12.2   12.2    ---  1017.4  0.00   0.0     0     0.00      0     0.042   0.000   23.3    45   10.7   23.0    0.00   232    1    100.0   10 
15/03/22   1:20   12.0   12.2   11.8    86    9.7   0.0   ---   0.00   0.0   ---   12.0   11.9   11.9    ---  1017.3  0.00   0.0     0     0.00      0     0.044   0.000   23.2    45   10.6   22.8    0.00   234    1    100.0   10 
15/03/22   1:30   11.7   11.8   11.6    87    9.6   0.0   ---   0.00   0.0   ---   11.7   11.7   11.7    ---  1017.3  0.00   0.0     0     0.00      0     0.046   0.000   23.1    45   10.5   22.7    0.00   234    1    100.0   10 
15/03/22   1:40   11.7   11.7   11.6    87    9.6   0.0   ---   0.00   0.0   ---   11.7   11.7   11.7    ---  1017.1  0.00   0.0     0     0.00      0     0.046   0.000   23.0    45   10.4   22.6    0.00   235    1    100.0   10 
15/03/22   1:50   11.7   11.8   11.6    87    9.6   0.0   ---   0.00   0.0   ---   11.7   11.7   11.7    ---  1017.2  0.00   0.0     0     0.00      0     0.046   0.000   22.9    45   10.3   22.4    0.00   234    1    100.0   10 
15/03/22   2:00   11.6   11.7   11.4    87    9.5   0.0   ---   0.00   0.0   ---   11.6   11.6   11.6    ---  1017.0  0.00   0.0     0     0.00      0     0.047   0.000   22.8    45   10.2   22.3    0.00   234    1    100.0   10 
15/03/22   2:10   11.5   11.6   11.4    88    9.6   0.0   ---   0.00   0.0   ---   11.5   11.6   11.6    ---  1017.0  0.00   0.0     0     0.00      0     0.047   0.000   22.7    45   10.1   22.1    0.00   231    1    100.0   10 
15/03/22   2:20   11.3   11.5   11.2    88    9.4   0.0   ---   0.00   0.0   ---   11.3   11.4   11.4    ---  1016.8  0.00   0.0     0     0.00      0     0.049   0.000   22.6    45   10.0   21.9    0.00   234    1    100.0   10 
15/03/22   2:30   11.2   11.3   11.2    88    9.3   0.0   ---   0.00   0.0   ---   11.2   11.3   11.3    ---  1016.9  0.00   0.0     0     0.00      0     0.049   0.000   22.4    45    9.9   21.8    0.00   234    1    100.0   10 
15/03/22   2:40   11.3   11.5   11.2    88    9.4   0.0   ---   0.00   0.0   ---   11.3   11.4   11.4    ---  1016.7  0.00   0.0     0     0.00      0     0.049   0.000   22.3    46   10.1   21.7    0.00   233    1    100.0   10 
15/03/22   2:50   11.6   11.6   11.4    87    9.5   0.0   ---   0.00   0.0   ---   11.6   11.6   11.6    ---  1017.0  0.00   0.0     0     0.00      0     0.047   0.000   22.2    46   10.0   21.5    0.00   233    1    100.0   10 
15/03/22   3:00   11.6   11.6   11.4    88    9.6   0.0   ---   0.00   0.0   ---   11.6   11.6   11.6    ---  1016.8  0.00   0.0     0     0.00      0     0.047   0.000   22.2    46   10.0   21.4    0.00   233    1    100.0   10 
15/03/22   3:10   11.6   11.6   11.5    87    9.5   0.0   ---   0.00   0.0   ---   11.6   11.6   11.6    ---  1016.5  0.00   0.0     0     0.00      0     0.047   0.000   22.1    46    9.9   21.2    0.00   234    1    100.0   10 
15/03/22   3:20   11.2   11.5   10.9    87    9.1   0.0   ---   0.00   0.0   ---   11.2   11.3   11.3    ---  1016.5  0.00   0.0     0     0.00      0     0.049   0.000   21.9    46    9.7   21.0    0.00   233    1    100.0   10 
15/03/22   3:30   10.9   11.1   10.8    88    9.0   0.0   ---   0.00   0.0   ---   10.9   11.0   11.0    ---  1016.5  0.00   0.0     0     0.00      0     0.051   0.000   21.8    46    9.6   20.8    0.00   234    1    100.0   10 
15/03/22   3:40   11.2   11.2   11.1    88    9.3   0.0   ---   0.00   0.0   ---   11.2   11.2   11.2    ---  1016.8  0.00   0.0     0     0.00      0     0.050   0.000   21.7    46    9.6   20.7    0.00   234    1    100.0   10 
15/03/22   3:50   11.2   11.3   11.2    88    9.3   0.0   ---   0.00   0.0   ---   11.2   11.3   11.3    ---  1016.7  0.00   0.0     0     0.00      0     0.049   0.000   21.6    46    9.5   20.6    0.00   233    1    100.0   10 
15/03/22   4:00   11.3   11.4   11.2    86    9.1   0.0   ---   0.00   0.0   ---   11.3   11.3   11.3    ---  1016.7  0.00   0.0     0     0.00      0     0.049   0.000   21.5    46    9.4   20.4    0.00   234    1    100.0   10 
15/03/22   4:10   11.2   11.2   11.1    87    9.1   0.0   ---   0.00   0.0   ---   11.2   11.2   11.2    ---  1016.7  0.00   0.0     0     0.00      0     0.050   0.000   21.4    46    9.3   20.3    0.00   234    1    100.0   10 
15/03/22   4:20   11.2   11.2   11.1    87    9.1   0.0   ---   0.00   0.0   ---   11.2   11.2   11.2    ---  1016.7  0.00   0.0     0     0.00      0     0.050   0.000   21.3    46    9.2   20.2    0.00   233    1    100.0   10 
15/03/22   4:30   11.1   11.2   10.9    87    9.0   0.0   ---   0.00   0.0   ---   11.1   11.1   11.1    ---  1016.9  0.00   0.0     0     0.00      0     0.051   0.000   21.2    46    9.1   20.1    0.00   234    1    100.0   10 
15/03/22   4:40   10.9   10.9   10.8    88    9.0   0.0   ---   0.00   0.0   ---   10.9   10.9   10.9    ---  1016.9  0.00   0.0     0     0.00      0     0.052   0.000   21.1    46    9.0   19.9    0.00   233    1    100.0   10 
15/03/22   4:50   10.8   10.9   10.7    87    8.8   0.0   ---   0.00   0.0   ---   10.8   10.9   10.9    ---  1016.8  0.00   0.0     0     0.00      0     0.052   0.000   21.0    47    9.3   19.9    0.00   234    1    100.0   10 
15/03/22   5:00   10.6   10.7   10.5    88    8.7   0.0   ---   0.00   0.0   ---   10.6   10.6   10.6    ---  1017.0  0.00   0.0     0     0.00      0     0.054   0.000   20.9    47    9.2   19.9    0.00   234    1    100.0   10 
15/03/22   5:10   10.6   10.7   10.5    88    8.7   0.0   ---   0.00   0.0   ---   10.6   10.7   10.7    ---  1017.1  0.00   0.0     0     0.00      0     0.054   0.000   20.9    47    9.2   19.8    0.00   233    1    100.0   10 
15/03/22   5:20   10.6   10.7   10.6    88    8.7   0.0   ---   0.00   0.0   ---   10.6   10.7   10.7    ---  1017.1  0.00   0.0     0     0.00      0     0.054   0.000   20.8    47    9.0   19.8    0.00   234    1    100.0   10 
15/03/22   5:30   10.6   10.7   10.6    89    8.8   0.0   ---   0.00   0.0   ---   10.6   10.7   10.7    ---  1017.3  0.00   0.0     0     0.00      0     0.054   0.000   20.7    47    8.9   19.7    0.00   232    1    100.0   10 
15/03/22   5:40   10.6   10.6   10.3    87    8.5   0.0   ---   0.00   0.0   ---   10.6   10.6   10.6    ---  1017.3  0.00   0.0     0     0.00      0     0.054   0.000   20.6    47    8.8   19.6    0.00   233    1    100.0   10 
15/03/22   5:50   10.3   10.3   10.2    89    8.5   0.0   ---   0.00   0.0   ---   10.3   10.4   10.4    ---  1017.7  0.00   0.0     0     0.00      0     0.056   0.000   20.5    47    8.8   19.6    0.00   234    1    100.0   10 
15/03/22   6:00   10.4   10.6   10.3    88    8.5   0.0   ---   0.00   0.0   ---   10.4   10.5   10.5    ---  1017.6  0.00   0.0     0     0.00      0     0.055   0.000   20.4    47    8.7   19.4    0.00   234    1    100.0   10 
15/03/22   6:10   10.6   10.7   10.3    86    8.3   0.0   ---   0.00   0.0   ---   10.6   10.6   10.6    ---  1017.5  0.00   0.0     0     0.00      0     0.054   0.000   20.3    47    8.6   19.3    0.00   235    1    100.0   10 
15/03/22   6:20   10.2   10.3   10.1    87    8.1   0.0   ---   0.00   0.0   ---   10.2   10.2   10.2    ---  1017.7  0.00   0.0     0     0.00      0     0.057   0.000   20.2    47    8.5   19.3    0.00   233    1    100.0   10 
15/03/22   6:30   10.0   10.1    9.8    87    7.9   0.0   ---   0.00   0.0   ---   10.0   10.1   10.1    ---  1018.0  0.00   0.0     0     0.00      5     0.058   0.000   20.1    47    8.4   19.2    0.00   234    1    100.0   10 
15/03/22   6:40    9.9    9.9    9.8    88    8.0   0.0   ---   0.00   0.0   ---    9.9   10.0   10.0    ---  1018.0  0.00   0.0     7     0.10      9     0.059   0.000   20.1    47    8.4   19.2    0.00   234    1    100.0   10 
15/03/22   6:50   10.1   10.4    9.9    89    8.4   0.0   ---   0.00   0.0   ---   10.1   10.2   10.2    ---  1018.2  0.00   0.0    16     0.23     26     0.057   0.000   20.0    47    8.3   19.1    0.00   234    1    100.0   10 
15/03/22   7:00   10.6   10.9   10.4    90    9.0   0.0   ---   0.00   0.0   ---   10.6   10.7   10.7    ---  1018.2  0.00   0.0    39     0.56     51     0.054   0.000   19.9    47    8.2   19.0    0.00   233    1    100.0   10 
15/03/22   7:10   11.3   11.7   10.9    88    9.4   0.0   ---   0.00   0.0   ---   11.3   11.3   11.3    ---  1018.2  0.00   0.0    65     0.93     77     0.049   0.000   19.8    48    8.5   19.0    0.00   232    1    100.0   10 
15/03/22   7:20   12.1   12.5   11.7    87   10.0   0.0   ---   0.00   0.0   ---   12.1   12.1   12.1    ---  1018.5  0.00   0.0    93     1.33    107     0.044   0.000   19.7    48    8.4   18.8    0.00   235    1    100.0   10 
15/03/22   7:30   13.1   13.6   12.6    83   10.2   0.0    SW   0.00   6.4    SW   13.1   12.9   12.9    ---  1018.3  0.00   0.0   126     1.81    142     0.037   0.000   19.7    48    8.3   18.8    0.00   233    1    100.0   10 
15/03/22   7:40   14.1   14.7   13.7    79   10.5   0.0   ---   0.00   0.0   ---   14.1   13.9   13.9    ---  1018.6  0.00   0.0   164     2.35    183     0.029   0.000   19.7    48    8.3   18.8    0.00   234    1    100.0   10 
15/03/22   7:50   15.2   15.6   14.7    76   11.0   0.0   ---   0.00   0.0   ---   15.2   14.9   14.9    ---  1018.6  0.00   0.0   202     2.90    218     0.022   0.000   19.6    48    8.2   18.7    0.00   232    1    100.0   10 
15/03/22   8:00   16.0   16.4   15.6    73   11.2   0.0   ---   0.00   0.0   ---   16.0   15.8   15.8    ---  1018.8  0.00   0.0   238     3.41    253     0.016   0.000   19.6    48    8.2   18.7    0.08   234    1    100.0   10 
15/03/22   8:10   16.8   17.1   16.4    73   11.9   0.0   WNW   0.00   3.2   WNW   16.8   16.6   16.6    ---  1018.8  0.00   0.0   271     3.88    285     0.011   0.000   19.4    48    8.1   18.6    0.00   234    1    100.0   10 
15/03/22   8:20   17.5   17.8   17.1    70   12.0   0.0   ---   0.00   0.0   ---   17.5   17.4   17.4    ---  1018.8  0.00   0.0   302     4.33    315     0.006   0.000   19.5    49    8.5   18.7    0.00   235    1    100.0   10 
15/03/22   8:30   18.2   18.6   17.8    71   12.8   0.0   ---   0.00   0.0   ---   18.2   18.2   18.2    ---  1018.8  0.00   0.0   332     4.76    346     0.001   0.000   19.6    49    8.5   18.7    0.00   233    1    100.0   10 
15/03/22   8:40   18.9   19.2   18.6    67   12.7   0.0   WNW   0.00   3.2   WNW   18.9   18.9   18.9    ---  1018.9  0.00   0.0   363     5.20    378     0.000   0.004   19.6    49    8.5   18.7    0.00   233    1    100.0   10 
15/03/22   8:50   19.5   19.8   19.2    67   13.2   0.0   WNW   0.00   1.6   WNW   19.5   19.6   19.6    ---  1018.8  0.00   0.0   395     5.66    408     0.000   0.008   19.6    49    8.5   18.7    0.00   231    1    100.0   10 
15/03/22   9:00   20.3   20.8   19.8    65   13.5   0.0   ---   0.00   0.0   ---   20.3   20.3   20.3    ---  1018.7  0.00   0.0   423     6.06    436     0.000   0.014   19.6    49    8.6   18.8    0.20   233    1    100.0   10 
15/03/22   9:10   21.2   21.6   20.8    63   13.9   0.0   ---   0.00   0.0   ---   21.2   20.9   20.9    ---  1018.6  0.00   0.0   452     6.48    466     0.000   0.020   19.7    49    8.6   18.8    0.00   234    1    100.0   10 
15/03/22   9:20   21.8   22.1   21.7    61   14.0   0.0   WNW   0.00   1.6   WNW   21.8   21.7   21.7    ---  1018.6  0.00   0.0   481     6.90    494     0.000   0.024   19.7    49    8.6   18.8    0.00   233    1    100.0   10 
15/03/22   9:30   22.4   22.7   22.2    59   14.0   0.0   WNW   0.00   3.2   WNW   22.4   22.4   22.4    ---  1018.4  0.00   0.0   508     7.28    519     0.000   0.029   19.7    50    9.0   18.9    0.00   235    1    100.0   10 
15/03/22   9:40   23.3   23.7   22.8    54   13.4   0.0   ---   0.00   0.0   ---   23.3   23.3   23.3    ---  1018.3  0.00   0.0   534     7.65    547     0.000   0.034   19.8    50    9.1   19.1    0.00   234    1    100.0   10 
15/03/22   9:50   24.4   25.1   23.7    47   12.3   0.0   WNW   0.00   1.6   WNW   24.4   24.3   24.3    ---  1018.3  0.00   0.0   562     8.06    575     0.000   0.042   20.0    51    9.6   19.3    0.00   233    1    100.0   10 
15/03/22  10:00   25.3   25.6   25.1    48   13.5   0.0   SSW   0.00   1.6   WSW   25.3   25.2   25.2    ---  1018.2  0.00   0.0   589     8.44    598     0.000   0.049   20.1    51    9.7   19.4    0.33   233    1    100.0   10 
15/03/22  10:10   25.9   26.1   25.6    44   12.7   0.0   ESE   0.00   6.4   ENE   25.9   25.7   25.7    ---  1018.2  0.00   0.0   612     8.77    624     0.000   0.052   20.2    51    9.8   19.6    0.00   234    1    100.0   10 
15/03/22  10:20   26.1   26.1   26.0    45   13.2   0.0   ESE   0.00   4.8   ESE   26.1   25.9   25.9    ---  1018.0  0.00   0.0   614     8.80    638     0.000   0.054   20.3    51    9.9   19.6    0.00   234    1    100.0   10 
15/03/22  10:30   26.2   26.3   26.1    46   13.6   1.6   ESE   0.27   8.0    SE   26.2   26.1   26.1    ---  1018.1  0.00   0.0   654     9.38    666     0.000   0.054   20.5    51   10.0   19.8    0.00   235    1    100.0   10 
15/03/22  10:40   26.3   26.4   26.3    45   13.4   1.6   ESE   0.27  11.3    SE   26.3   26.2   26.2    ---  1018.1  0.00   0.0   676     9.69    684     0.000   0.056   20.7    50    9.9   19.9    0.00   233    1    100.0   10 
15/03/22  10:50   26.4   26.6   26.3    47   14.2   1.6   ESE   0.27   9.7   ESE   26.4   26.3   26.3    ---  1018.0  0.00   0.0   694     9.95    703     0.000   0.056   20.9    50   10.1   20.0    0.00   234    1    100.0   10 
15/03/22  11:00   27.0   27.3   26.6    45   14.1   0.0     N   0.00   4.8     N   27.0   26.8   26.8    ---  1017.7  0.00   0.0   712    10.21    717     0.000   0.060   21.1    50   10.2   20.1    0.43   234    1    100.0   10 
15/03/22  11:10   27.1   27.4   26.9    42   13.1   1.6   ESE   0.27   8.0   ESE   27.1   26.7   26.7    ---  1017.6  0.00   0.0   728    10.44    735     0.000   0.061   21.3    50   10.4   20.4    0.00   233    1    100.0   10 
15/03/22  11:20   27.0   27.1   26.9    43   13.4   3.2   ESE   0.54   8.0   ESE   27.0   26.7   26.7    ---  1017.6  0.00   0.0   741    10.62    749     0.000   0.060   21.5    49   10.3   20.6    0.00   233    1    100.0   10 
15/03/22  11:30   27.2   27.6   27.0    42   13.1   0.0   ESE   0.00   8.0   ESE   27.2   26.7   26.7    ---  1017.5  0.00   0.0   754    10.81    759     0.000   0.061   21.6    49   10.4   20.8    0.00   233    1    100.0   10 
15/03/22  11:40   27.8   27.9   27.6    38   12.2   0.0    NW   0.00   9.7   NNW   27.8   27.3   27.3    ---  1017.5  0.00   0.0   767    11.00    773     0.000   0.066   21.8    49   10.6   21.1    0.00   235    1    100.0   10 
15/03/22  11:50   28.3   28.4   27.9    37   12.2   0.0    SE   0.00   3.2    NW   28.3   27.8   27.8    ---  1017.2  0.00   0.0   779    11.17    784     0.000   0.069   22.1    49   10.8   21.4    0.00   233    1    100.0   10 
15/03/22  12:00   28.2   28.5   27.6    37   12.1   4.8   ESE   0.80  11.3   ESE   28.2   27.7   27.7    ---  1017.3  0.00   0.0   786    11.27    788     0.000   0.069   22.2    48   10.7   21.6    0.51   234    1    100.0   10 
15/03/22  12:10   27.6   28.0   27.4    40   12.8   0.0   ESE   0.00   6.4   ESE   27.6   27.2   27.2    ---  1017.0  0.00   0.0   791    11.34    795     0.000   0.064   22.4    48   10.8   21.8    0.00   234    1    100.0   10 
15/03/22  12:20   28.5   28.8   28.1    36   12.0   0.0   ---   0.00   0.0   ---   28.5   28.0   28.0    ---  1016.9  0.00   0.0   795    11.40    800     0.000   0.071   22.6    47   10.7   22.1    0.00   234    1    100.0   10 
15/03/22  12:30   29.0   29.1   28.8    36   12.4   0.0   ESE   0.00   1.6     E   29.0   28.7   28.7    ---  1016.7  0.00   0.0   800    11.47    802     0.000   0.074   22.8    47   10.9   22.4    0.00   234    1    100.0   10 
15/03/22  12:40   29.1   29.2   29.1    36   12.5   0.0   ENE   0.00   1.6   ESE   29.1   28.8   28.8    ---  1016.6  0.00   0.0   802    11.50    803     0.000   0.075   22.9    46   10.7   22.6    0.00   235    1    100.0   10 
15/03/22  12:50   29.1   29.2   29.1    34   11.6   0.0     E   0.00   1.6     E   29.1   28.6   28.6    ---  1016.4  0.00   0.0   802    11.50    803     0.000   0.075   23.2    46   10.9   22.8    0.00   234    1    100.0   10 
15/03/22  13:00   29.1   29.3   29.1    37   12.9   0.0   WNW   0.00   1.6   WNW   29.1   28.9   28.9    ---  1016.4  0.00   0.0   798    11.44    800     0.000   0.075   23.4    46   11.1   23.1    0.53   234    1    100.0   10 
15/03/22  13:10   29.6   29.7   29.4    37   13.4   0.0     W   0.00   8.0     W   29.6   29.4   29.4    ---  1016.1  0.00   0.0   795    11.40    798     0.000   0.078   23.6    46   11.3   23.3    0.00   234    1    100.0   10 
15/03/22  13:20   29.6   29.7   29.4    35   12.5   0.0     W   0.00   6.4     W   29.6   29.1   29.1    ---  1015.8  0.00   0.0   790    11.32    793     0.000   0.078   23.8    45   11.1   23.6    0.00   233    1    100.0   10 
15/03/22  13:30   29.5   29.6   29.4    34   12.0   0.0    NW   0.00   4.8    NW   29.5   28.9   28.9    ---  1015.7  0.00   0.0   783    11.22    788     0.000   0.078   23.9    45   11.3   23.8    0.00   233    1    100.0   10 
15/03/22  13:40   29.6   29.6   29.5    35   12.5   1.6     W   0.27   9.7   WSW   29.6   29.1   29.1    ---  1015.5  0.00   0.0   773    11.08    779     0.000   0.078   24.2    45   11.5   23.9    0.00   234    1    100.0   10 
15/03/22  13:50   29.7   29.8   29.6    36   13.0   1.6     W   0.27   8.0     W   29.7   29.3   29.3    ---  1015.5  0.00   0.0   762    10.92    768     0.000   0.079   24.3    44   11.3   24.1    0.00   235    1    100.0   10 
15/03/22  14:00   29.7   29.8   29.6    34   12.1   1.6   SSW   0.27   6.4   SSW   29.7   29.1   29.1    ---  1015.2  0.00   0.0   750    10.75    756     0.000   0.079   24.5    44   11.4   24.2    0.53   233    1    100.0   10 
15/03/22  14:10   29.7   29.8   29.6    34   12.1   0.0    SW   0.00   6.4    SW   29.7   29.1   29.1    ---  1015.0  0.00   0.0   736    10.55    745     0.000   0.079   24.7    44   11.6   24.4    0.00   234    1    100.0   10 
15/03/22  14:20   29.8   29.9   29.6    34   12.2   0.0    SW   0.00   6.4    SW   29.8   29.3   29.3    ---  1014.8  0.00   0.0   572     8.20    724     0.000   0.079   24.8    43   11.4   24.5    0.00   234    1    100.0   10 
15/03/22  14:30   29.7   30.0   29.6    35   12.6   0.0    SW   0.00   3.2    SW   29.7   29.3   29.3    ---  1014.6  0.00   0.0   104     1.49    279     0.000   0.079   25.0    43   11.5   24.7    0.00   234    1    100.0   10 
15/03/22  14:40   30.2   30.3   30.0    33   12.1   0.0   WNW   0.00   4.8   WNW   30.2   29.6   29.6    ---  1014.4  0.00   0.0    36     0.52     37     0.000   0.082   25.2    43   11.7   24.8    0.00   234    1    100.0   10 
15/03/22  14:50   30.5   30.9   30.3    34   12.9   0.0     W   0.00   1.6     W   30.5   29.9   29.9    ---  1014.3  0.00   0.0    35     0.50     37     0.000   0.084   25.3    42   11.5   24.9    0.00   235    1    100.0   10 
15/03/22  15:00   31.0   31.2   30.8    32   12.4   0.0     W   0.00   1.6     W   31.0   30.0   30.0    ---  1014.2  0.00   0.0    37     0.53     37     0.000   0.088   25.5    42   11.6   25.1    0.18   234    1    100.0   10 
15/03/22  15:10   30.6   30.8   30.6    34   13.0   0.0     W   0.00   3.2     W   30.6   30.1   30.1    ---  1013.8  0.00   0.0    37     0.53     37     0.000   0.085   25.6    42   11.7   25.2    0.00   234    1    100.0   10 
15/03/22  15:20   30.8   30.9   30.6    31   11.7   0.0    SW   0.00   1.6   SSW   30.8   29.8   29.8    ---  1013.7  0.00   0.0    35     0.50     35     0.000   0.087   25.8    42   11.9   25.4    0.00   233    1    100.0   10 
15/03/22  15:30   30.8   30.9   30.7    34   13.1   0.0     W   0.00   4.8     W   30.8   30.1   30.1    ---  1013.5  0.00   0.0    35     0.50     35     0.000   0.086   25.9    42   12.0   25.6    0.00   234    1    100.0   10 
15/03/22  15:40   30.6   30.7   30.6    34   12.9   0.0   WNW   0.00   6.4   WNW   30.6   30.0   30.0    ---  1013.2  0.00   0.0    39     0.56     56     0.000   0.085   26.1    42   12.2   25.8    0.00   234    1    100.0   10 
15/03/22  15:50   30.8   31.1   30.7    32   12.2   0.0   ---   0.00   0.0   ---   30.8   29.9   29.9    ---  1013.1  0.00   0.0   158     2.26    271     0.000   0.087   26.3    41   12.0   25.8    0.00   233    1    100.0   10 
15/03/22  16:00   31.4   31.7   31.1    32   12.7   0.0   ---   0.00   0.0   ---   31.4   30.4   30.4    ---  1013.1  0.00   0.0   396     5.68    461     0.000   0.091   26.4    41   12.1   25.9    0.10   235    1    100.0   10 """, regex = inta_regex, date_format=inta_format)

print(model.predict(x))

[[0.02034476]]
